These financial data are generally provided in a format that includes the following information:

Date
Open Price
High Price
Low Price
Closing Price
Volume
These data—often referred to as OHLC Chart Data—can be interpreted as Time Series data and are perfect for performing technical analysis. We’ll dive into this format in just a moment but, for now, just realize this is a standard format for historical pricing data within financial markets.

yfinance: 
For this approach, we need to install the yfinance library as pip install yfinance. This library provides sample tools for working with financial data requests to the Yahoo Finance website. Keep in mind, however, this is not an official API and is subject to rate limiting, periodic breakage, and general quirkiness. Nonetheless, its the defacto Python library for OHLC data and can be used as follows:

In [10]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [11]:
import yfinance as yf
tick = ['NVDA','MSFT','AAC','AACG', 'AACI','AACIW','AADI','AAIC','AAL','AAMC','AAME','AAN','AAOI','AAON','AAP','AAPL','AAQC','AAT','AATC','AAU','AAWW','AB','ABB','ABBV','ABC']
for i in tick:
    data = yf.Ticker(i).history(period='5y')
    data.to_csv(i+".csv")

KeyboardInterrupt: 

Quandl:
Quandl offers official APIs to access any public dataset for free. Here we’ll see how to get OHLC data via the official Quandl python library and also via the pandas-datareader. One important note is that the free Quandl OHLC data only goes up to 2018 at the time of this article’s writing. If you need more recent data and don’t want to pay this source isn’t for you.

Quandl Python Library
To get started with Quadl’s official API we need to install the python library as such: pip install quandl. This will install the official quandl python library and let us make up to 50 daily API requests without registering an account. Let’s get our financial data:

In [ ]:
pip install quandl

     -------------------------------------- 52.2/52.2 kB 676.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import quandl
import time
tick = ['NVDA','MSFT','AAC','AACG', 'AACI','AACIW','AADI','AAIC','AAL','AAMC','AAME','AAN','AAOI','AAON','AAP','AAPL','AAQC','AAT','AATC','AAU','AAWW','AB','ABB','ABBV','ABC']
# Get data via Quandl API
j=0
for i in tick:
    j+=1
    k=1
    if j>2*k+1:
        time.sleep(600)
        k+=20
    data = quandl.get('WIKI/NVDA')
    data.to_csv(i+".csv")



after 10 sec


KeyboardInterrupt: 

Alpha Vantage:
The pandas-datareader library offers easy access to OHLC data via Alpha Vantage integration. The following code will retrieve historical data for different tickers once again:

In [ ]:
pip install alpha_vantage

     -------------------------------------- 554.9/554.9 kB 2.7 MB/s eta 0:00:00
     -------------------------------------- 56.7/56.7 kB 329.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
# Create an API object
ts = TimeSeries(key='UNO4CZQHSBZSN71N')
# Get daily OHLC data for NVDA
data, meta_data = ts.get_daily(symbol="NVDA")
data_df = pd.DataFrame(data)
data_df.to_csv("NVDA.csv", header=False)

Alpha Vantage:
TIME_SERIES_DAILY_ADJUSTED Premium Trending

This API returns raw (as-traded) daily open/high/low/close/volume values, daily adjusted close values, and historical split/dividend events of the global equity specified, covering 20+ years of historical data.


API Parameters
❚ Required: function

The time series of your choice. In this case, function=TIME_SERIES_DAILY_ADJUSTED

❚ Required: symbol

The name of the equity of your choice. For example: symbol=IBM

❚ Optional: outputsize

By default, outputsize=compact. Strings compact and full are accepted with the following specifications: compact returns only the latest 100 data points; full returns the full-length time series of 20+ years of historical data. The "compact" option is recommended if you would like to reduce the data size of each API call.

❚ Optional: datatype

By default, datatype=json. Strings json and csv are accepted with the following specifications: json returns the daily time series in JSON format; csv returns the time series as a CSV (comma separated value) file.

❚ Required: apikey

Your API key


In [ ]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&apikey=1NSFEG3NTR9VZZOF'
r = requests.get(url)
data = r.json()

print(data)

{'Information': 'Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints'}


Zerodha-historical-data-download :
This contains code to download historical data for more than 2000 days and intraday data for more than 100 days. Download the framework and customize accordingly.

The changes you need to make.

Set your working directory where you want your data to be downloaded.

Enter your API key.

Enter youy API secret.

Generate request_token.

Set the Access token.

In the 'tickers' variable write all the stock names you want(write the names exactly as in zerodha app).

in the fetchOHLC func, enter the number of days.

Note: If you want F&O data, change the instruments('NSE') to instruments('NFO').

In [ ]:
!pip install kiteconnect --upgrade --quiet

In [ ]:
from kiteconnect import KiteConnect, KiteTicker
import pandas as pd
import datetime as dt
import numpy as np
import os
import time
import json



api_key = 'your_api_key'
api_secret = 'your_api_secret'

kite = KiteConnect(api_key)
url = kite.login_url()

request_token = 'your_request_token'
session = kite.generate_session(request_token, api_secret)
access_token = session['access_token']
kite.set_access_token(access_token)

instruments = pd.DataFrame(kite.instruments('NSE'))


def lookup(df, symbol):
    try:
        return df[df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

def fetchandstoreOHLC(symbol,duration,tf):
    nodata=[]
    try:
        if not os.path.isfile(symbol+'.csv'):
            data = pd.DataFrame()
            instrument_token = lookup(instruments, symbol)  
            if duration>100:
                no_of_hundreds = duration//100
                start = 100
                end = 0
                for i in range(no_of_hundreds):
                    df = pd.DataFrame(kite.historical_data(instrument_token, from_date = dt.datetime.today()-dt.timedelta(start), 
                    to_date = dt.datetime.today()-dt.timedelta(end), interval = tf))
                    data = data.append(df)
                    start+=100
                    end+=100
                dur1 = duration%100
                df1 = pd.DataFrame(kite.historical_data(instrument_token, from_date = dt.datetime.today()-dt.timedelta(duration), 
                                      to_date = dt.datetime.today()-dt.timedelta(duration-dur1), interval = tf))
                data = data.append(df1)
            else:
                df2 = pd.DataFrame(kite.historical_data(instrument_token, from_date = dt.datetime.today()-dt.timedelta(duration), 
                                      to_date = dt.datetime.today(), interval = tf))
                data = data.append(df2)
            data.set_index("date", inplace = True)
            data.sort_index(ascending= True,inplace = True)
            print('Storing {} days data for {}'.format(duration, symbol))
            data.to_csv(symbol+'.csv')
        else:
            print('Data already available for{}'.format(symbol))
            return
    except:
        print("skipping for {}".format(symbol))
        nodata.append(symbol)
    
tickers = ['HDFCBANK','ICICIBANK','KOTAKBANK', 'AXISBANK', 'SBIN', 'RELIANCE','TCS','INFY','HINDUNILVR','HDFC','BAJFINANCE','WIPRO','BHARTIARTL','HCLTECH','ASIANPAINT','ITC','LT','ULTRACEMCO',
           'MARUTI','SUNPHARMA','TATASTEEL','JSWSTEEL','TITAN','ADANIPORTS','ONGC','HDFCLIFE','TECHM','DIVISLAB','POWERGRID','SBILIFE','NTPC','BAJAJ-AUTO','BPCL','IOC','M&M','SHREECEM','HINDALCO',
           'GRASIM','BRITANNIA','TATAMOTORS','COALINDIA','TATACONSUM','INDUSINDBK','DRREDDY','CIPLA','EICHERMOT','UPL','NESTLEIND','HEROMOTOCO','NIFTY 50','NIFTY BANK']


for ticker in tickers:
    fetchandstoreOHLC(ticker, 1095, '5minute')


TokenException: Token is invalid or has expired.

Angel One:
SMARTAPI-PYTHON is a Python library for dealing AMX,that is a set of REST-like HTTP APIs that expose many capabilities required to build stock market investment and trading platforms. It lets you execute orders in real time.


In [ ]:
!pip install smartapi-python --quiet

In [ ]:
from smartapi.smartConnect import SmartConnect
import pandas as pd
import requests
feed_token = None

obj = SmartConnect(api_key="api_key")
data = obj.generateSession("username", "password")
refreshToken=data['data']['refreshToken']

#fetch the feedtoken
feedToken = obj.getfeedToken()
feed_token = feedToken
#fetch User Profile
userProfile = obj.getProfile((refreshToken))
print(userProfile)

#historic api
def OHLCHistory(symbol, token, interval, fdate, todate):
    try:
        historicParam={
            "exchange":"NSE",
            "tradingsymbol":symbol,
            "symboltoken":token,
            "interval":interval,
            "fromdate":fdate,
            "todate":todate
        }
        history = obj.getCandleData(historicParam)['data']
        history = pd.DataFrame(history)

        history = history.rename(
            columns={0: "DateTime", 1: "open", 2: "high", 3: "low", 4: "close", 5:"Volumne"}
        )
        history['DateTime']=pd.to_datetime(history['DataTime'])
        history=history.set_index('DateTime')
        return history
    except Exception as e:
        print("Histpric api failed: {}".format(e))
data = OHLCHistory("SBIN-EQ", "3045", "ONE_DAY", "2020-02-08 00:00", "2021-02-08 15:30")
data_df = pd.DataFrame(data)
data_df.to_csv(r'{}.csv'.format("SBIN_EQ"), index=True, header=False)

ModuleNotFoundError: No module named 'smartapi'